# Maps

## Setup

In [33]:
# Import dependencies
import pandas as pd


from plotly.io import write_image, write_json
import plotly.express as px
import plotly.graph_objs as go

## Preprocessing Data

In [34]:
# Read in data from Excel file
df = pd.read_excel("shanhai.xlsx", header=0)

# Calculate km from li
df['distance_km'] = df['distance_li'] / 2

# Iterate through dataframe and calculate distance_lat and distance_lon for each row based on distance_km, depending on direction
# If direction is N, then distance_lat = distance_km / 111.3 and distance_lon = 0
# If direction is S, then distance_lat = -distance_km / 111.3 and distance_lon = 0
# If direction is E, then distance_lat = 0 and distance_lon = distance_km / 111.32
# If direction is W, then distance_lat = 0 and distance_lon = -distance_km / 111.32
for index, row in df.iterrows():
    if row['direction'] == 'N':
        df.loc[index, 'distance_lat'] = row['distance_km'] / 111.3
        df.loc[index, 'distance_lon'] = 0
    elif row['direction'] == 'S':
        df.loc[index, 'distance_lat'] = -row['distance_km'] / 111.3
        df.loc[index, 'distance_lon'] = 0
    elif row['direction'] == 'E':
        df.loc[index, 'distance_lat'] = 0
        df.loc[index, 'distance_lon'] = row['distance_km'] / 111.32
    elif row['direction'] == 'W':
        df.loc[index, 'distance_lat'] = 0
        df.loc[index, 'distance_lon'] = -row['distance_km'] / 111.32

# Update lat and lon for each row with distance_lat and distance_lon
for index, row in df.iterrows():
    if index == 0:
        df.loc[index, 'lat'] = row['lat']
        df.loc[index, 'lon'] = row['lon']
    else:
        df.loc[index, 'lat'] = df.loc[index-1, 'lat'] + row['distance_lat']
        df.loc[index, 'lon'] = df.loc[index-1, 'lon'] + row['distance_lon']

df

,lat,lon,place,distance,distance_li,direction_zh,direction,distance_lat,distance_lon,distance_km
0,30.0,107.000000,招搖之山,三百里,300,東,E,0.0,1.347467,150.0
1,30.0,108.706791,堂庭之山,三百八十里,380,東,E,0.0,1.706791,190.0
2,30.0,110.368667,猨翼之山,三百七十里,370,東,E,0.0,1.661876,185.0


## Plotting

In [35]:
### Basic example with Plotly Go ###

# Create figure data
fig = go.Figure(data=go.Scattergeo(
        lon = df['lon'],
        lat = df['lat'],
        text = df['place'],
        mode = 'markers+lines',
        # marker_color = df['cnt'],
        ))

# Update layout
fig.update_layout(
        title = 'Title',
        geo_scope='asia',
    )

# Show figure
fig.show()

### Settings for fancy maps

In [36]:
# Visual variables for map (light mode) ################
transparent = 'rgba(255,255,255,0)'
half_transparent = 'rgba(255,255,255,0.5)'
quarter_transparent = 'rgba(255,255,255,0.25)'
tenth_transparent = 'rgba(255,255,255,0.1)'

font_size = 16
font_color = "black"
font_family = "Sans-Serif"

marker_symbol= 'circle'
marker_size = 20
max_marker_size = 40

edge_color = 'white'
edge_size = 1
opacity = 0.7

line_width = 4

water = 'white'
grid_color = '#ededed'
land = 'gainsboro'
lines = 'gainsboro'

background_color = transparent
legend_background_color = tenth_transparent

######################################
# Visual variables for map (dark mode)

# transparent = 'rgba(0,0,0,0)'
# half_transparent = 'rgba(0,0,0,0.5)'
# quarter_transparent = 'rgba(0,0,0,0.25)'
# tenth_transparent = 'rgba(0,0,0,0.1)'

# font_size = 14
# font_color = "black"
# font_family = "Serif"

# marker_symbol= 'circle'
# marker_size = 14
# max_marker_size = 32

# edge_color = transparent
# edge_size = 1
# opacity = 0.7

# line_width = 4

# water = '#212f3c'
# grid_color = '#283747'
# land = ' #2e4053'
# lines = '#34495e'
# copyright_color = '#5d6d7e'

# background_color = transparent
# legend_background_color = tenth_transparent



######################################

# Colors
# https://plotly.com/python/discrete-color/ 
# https://plotly.com/python/builtin-colorscales/

# print("Prism colors:", px.colors.qualitative.Prism)

color_scheme = px.colors.qualitative.Prism


In [37]:
# Orthographic globe layout
ortho_traces = dict(
    textposition = 'top right', # middle left, bottom center, etc.
    textfont = dict(size=font_size, color=font_color, family=font_family),
    # hovertemplate=
        # "<b>%{text}</b><br><br>" +
        # "Species: <i>%{customdata[1]}</i><br>" +
        # "Family: <i>%{customdata[2]}</i><br>" +
        # "Region of origin: %{customdata[3]}<br>" +
        # "Arabic: %{customdata[4]} <i>%{customdata[5]}</i><br>" +
        # "Chinese: %{customdata[6]} <i>%{customdata[7]}</i><br>" +
        # "Spreadability: %{customdata[8]:.2f}<br>" +
        # "<extra></extra>",
    marker = dict(
        symbol = marker_symbol,
        size = marker_size,
        line = dict(
            color=edge_color,
            width=edge_size
        )
    )
)

ortho_layout = go.Layout(
    paper_bgcolor=background_color,
    plot_bgcolor=background_color,
    geo = dict(
        resolution=110, # 50 is large; 110 is small
        scope='world', # 'asia'
        projection_type = 'orthographic',
        projection_scale = 1,
        projection_rotation = {'lat': 15, 'lon': 30, 'roll': 0},
        bgcolor=background_color,
        showcoastlines=True, coastlinewidth = 1, coastlinecolor = lines,
        showcountries=False, countrywidth = 1, countrycolor = lines, 
        showframe=True, framewidth = 1, framecolor = lines, 
        showlakes=True, lakecolor = water,
        showland=True, landcolor = land, 
        showocean=True, oceancolor = water,
        showrivers=True, riverwidth = 1, rivercolor = water,
        showsubunits=False, subunitwidth = 1, subunitcolor = lines, 
        lonaxis = dict(showgrid = True, gridwidth = 0.5, dtick = 10, gridcolor=grid_color),
        lataxis = dict (showgrid = True, gridwidth = 0.5, dtick = 10, gridcolor=grid_color)),
    showlegend = True,
    legend=dict(x=0, y=0, xanchor="left", yanchor="bottom", bgcolor=half_transparent,  
                font=dict(color=font_color, size=font_size, family=font_family), 
                title_font=dict(color=font_color, size=font_size+2, family=font_family),
                traceorder = 'normal', orientation="v"),
    title=dict(x=0.5, y=0.99, xanchor='center', yanchor='top', text='',
               font=dict(color=font_color, size=font_size+6, family=font_family)),
    margin={"r":0,"t":0,"l":0,"b":0},
    hoverlabel=dict(#bgcolor="white", 
                    font_size=font_size, 
                    font_family=font_family),
    )

# "Document size" for pdfs
document_size = dict(width = 600, height=600)

In [38]:
# Image (PNG/PDF) for documents

# Set size
df['size'] = 1

# Create figure data
data = px.scatter_geo(df,
    lat='lat', 
    lon='lon',
    text='place',
    color='place',
    color_discrete_sequence=color_scheme,
    size_max = max_marker_size,
    size = 'size',
    opacity = opacity,
    hover_name='place',
    # hover_data={'url':False, 'species':True, 'family':True, 'region of origin':True, 'Arabic':True, 'Ar transliteration':True, 'Chinese':True, 'pinyin':True, 'spreadability':':.2f', 'lon':False, 'lat':False, 'size':False},
    # labels={"group": "category"}
    )

# Save figure data
fig = data

# Call the orthographic traces and layout settings from above
fig.update_traces(ortho_traces)
fig.update_layout(ortho_layout)
fig.update_layout(geo=dict(projection_rotation = {'lat': 20, 'lon': 80, 'roll': 0}))
fig.update_layout(document_size)

# Show figure
fig.show()

# Write image (PNG/PDF) for documents
filename = "test"
fig.write_image(filename + ".pdf", engine="kaleido")
fig.write_image(filename + ".png", scale=3)

################################################################################
# Interactive visualization (HTML/JSON) for the web

# Call figure data
fig=data

# Call the orthographic traces and layout settings from above
fig.update_traces(ortho_traces)
fig.update_layout(ortho_layout)
fig.update_layout(geo=dict(projection_rotation = {'lat': 20, 'lon': 80, 'roll': 0}))
# fig.update_layout(title_text = "Title")

# Show figure
# fig.show()

# Write interactive visualization (HTML/JSON) for the web
filename = "test"
fig.write_html(filename + ".html")
fig.write_json(filename + ".json", validate=True, pretty=True)

# Notes

In [39]:
# A basic example with Plotly Express
import plotly.express as px
fig = px.scatter_geo(df, lat="lat", lon='lon', color="direction",
                     hover_name="place", size="distance_km",
                     projection="natural earth")
fig.show()